# Training an ANN to Respond to Simple Questions

In this notebook I'll try to use Keras to train a SEQ2SEQ model which will hopefully give some bearable answers to my questions. I'll play with some random codes from GitHub then write my own improwed model.

## 1. Preparations

In [1]:
def decode(sequence, lookup, separator=''): # 0 used for padding, is ignored
    return separator.join([ lookup[element] for element in sequence if element ])

In [2]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np
import pickle
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [3]:
batch_size = 64  # Batch size for training.
epochs = 150  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 20000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'data/twitter_dialogue.csv'

In [4]:
# Vectorize the data.
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

In [5]:
data = pd.read_csv(data_path)
for idx in range(min(len(data), num_samples)):
    input_text, target_text = data['question'][idx], data['answer'][idx]
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    if not pd.isnull(target_text) and not pd.isnull(input_text):
        target_text = '\t' + target_text + '\n'
        input_texts.append(input_text)
        target_texts.append(target_text)
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)

In [6]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 19990
Number of unique input tokens: 37
Number of unique output tokens: 39
Max sequence length for inputs: 131
Max sequence length for outputs: 128


In [7]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [8]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [9]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

## 2. Building the Model

In [10]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

Instructions for updating:
Colocations handled automatically by placer.


In [11]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [12]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [13]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

## 3. Training the Model

In [14]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs, verbose=1, validation_split=0.1)
# Save model
model.save('s2s_twitter_20000.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 17991 samples, validate on 1999 samples
Epoch 1/150
17991/17991 [==============================] - 47s 3ms/step - loss: 0.9421 - val_loss: 0.8299
Epoch 2/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.7921 - val_loss: 0.7356
Epoch 3/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.7233 - val_loss: 0.6840
Epoch 4/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.6770 - val_loss: 0.6581
Epoch 5/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.6426 - val_loss: 0.6239
Epoch 6/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.6162 - val_loss: 0.5994
Epoch 7/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.5949 - val_loss: 0.5890
Epoch 8/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.5

17991/17991 [==============================] - 45s 3ms/step - loss: 0.3954 - val_loss: 0.5711
Epoch 73/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3948 - val_loss: 0.5725
Epoch 74/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3937 - val_loss: 0.5743
Epoch 75/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3929 - val_loss: 0.5755
Epoch 76/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3922 - val_loss: 0.5778
Epoch 77/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3916 - val_loss: 0.5771
Epoch 78/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3910 - val_loss: 0.5789
Epoch 79/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3902 - val_loss: 0.5790
Epoch 80/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3897 - val_loss: 0.5799
Epoch 81/150
17991/17991 [========================

17991/17991 [==============================] - 45s 3ms/step - loss: 0.3650 - val_loss: 0.6216
Epoch 149/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3649 - val_loss: 0.6171
Epoch 150/150
17991/17991 [==============================] - 45s 3ms/step - loss: 0.3647 - val_loss: 0.6220


In [15]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [16]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [17]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

## 4. Testing 

In [18]:
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: yeah im preparing myself to drop a lot on this man but definitely need something reliable
Decoded sentence: thats a  the  of the review its a big  suit right now 

-
Input sentence: shouldnt the supporters natural answer to s hashtag be 
Decoded sentence: the  in the  in the  of the day they were coming out 

-
Input sentence: you want to turn twitter followers into blog readers
Decoded sentence: thats a  the  of the day that  to  and  every match the clinton

-
Input sentence: well i finally finished watching all the episodes of breaking the  code  biggest secrets finally revealed on netflix
Decoded sentence: it was a man be surprise  i love it thanks

-
Input sentence: does the dog also have 
Decoded sentence: i want to know what the hell outsident concert in the  the    and  

-
Input sentence:  operation in  best response for any action pakistan and army
Decoded sentence:             

-
Input sentence:   shots 
Decoded sentence:           

-
Input sentence:  is 

-
Input sentence: the social manager in me is   many brands looked up to the lil candy  
Decoded sentence: that was the  and card the beautiful this second that i fucking here than 

-
Input sentence: why does this screen use so little of my my display
Decoded sentence: i want to know what this way and it was  

-
Input sentence: did someone pee in ur cereal this morning u look amp talk  u should retire
Decoded sentence: i was just think you shouldnt be sure the started it in the base

-
Input sentence: i just really want to know is beating down a group of bears this badly considered a hate crime
Decoded sentence: i want to know what the hell outsident concert in the  the    and  

-
Input sentence: this is so real
Decoded sentence: the  is the  is to help

-
Input sentence:  android phones wont have headphone jokes in the next 18 months
Decoded sentence:  is the best part im at  

-
Input sentence:  ye  it is almost time for a perfect day  it
Decoded sentence: i was just think you sho

**Final thoughts:** works, but not accurate and gives incorrect responses. The structure of the model is good but the data has to be cleaned or even changed.